In [1]:
import joblib
import pandas as pd
import unicodedata
import re
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
pipeline_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/pipeline/randomforestmurspipeline.pkl"
pipeline = joblib.load(pipeline_path)

In [ ]:
# Charger le CSV
csv_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/datatest/test6/cleantest6/mursclean6.csv"
if not os.path.isfile(csv_path):
    print(f"Le fichier n'existe pas : {csv_path}")
    df_test = None
else:
    df_test = pd.read_csv(csv_path, delimiter=';', header=1)

# Nettoyer les noms de colonnes comme à l'entraînement
def clean_col(col):
    col = ''.join(c for c in unicodedata.normalize('NFD', col) if unicodedata.category(c) != 'Mn')
    col = col.lower()
    col = re.sub(r"[ \-\(\)]", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col)
    col = col.strip("_")
    return col

if df_test is not None:
    df_test.columns = [clean_col(c) for c in df_test.columns]

In [4]:
df_test

,id,011ec_lot,012ec_ouvrage,013ec_localisation,014ec_mode_constructif,nom,hauteur,epaisseur,ai,as,...,ifcexporttype,fisa_code1,renovation_status,servicelife,acousticrating,combustible,thermaltransmittance,group,sub_group,marque_centrale_visible
0,7257435,NaN,NaN,NaN,NaN,VPP 250,"3,35000000000007","0,25","-8,16999999999973","-4,81999999999966",...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN
1,7257572,NaN,NaN,NaN,NaN,VPP 250,"3,34999999999999","0,25","-8,16999999999973","-4,81999999999974",...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN
2,7257587,NaN,NaN,NaN,NaN,VPP 250,"3,35000000000007","0,25","-8,16999999999973","-4,81999999999966",...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN
3,7257602,NaN,NaN,NaN,NaN,VPP 250,"3,26660000000006","0,25","-8,16999999999972","-4,90339999999966",...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN
4,7257617,NaN,NaN,NaN,NaN,VPP 250,"3,15000000000005","0,25","-8,16999999999972","-5,01999999999966",...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,8670242,NaN,NaN,NaN,NaN,Mur Béton 200 mm,"2,06880121111507","0,2","-9,67299999999972","-7,60419878888465",...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN
1262,8670329,NaN,NaN,NaN,NaN,Mur Béton 200 mm,"2,06880121111507","0,2","-9,67299999999972","-7,60419878888465",...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN
1263,8674390,NaN,NaN,NaN,NaN,Mur Béton 250 mm,"2,20441035822074","0,25","-6,46999999999973","-4,26558964177899",...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN
1264,8825173,NaN,NaN,NaN,NaN,Mur Béton 200 mm,"0,947076211115074","0,2","-8,55127499999973","-7,60419878888465",...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN


In [5]:
# Les targets à prédire (après nettoyage)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Colonnes explicatives attendues par le pipeline
features = pipeline.named_steps['preprocessor'].feature_names_in_

# S'assurer que toutes les colonnes sont présentes
for col in features:
    if col not in df_test.columns:
        df_test[col] = np.nan
        print(f"Colonne manquante ajoutée : {col}")
X_test = df_test[features].copy()


In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
y_pred

array([['GO', 'MUR', 'INTERIEUR', 'BANCHE'],
       ['GO', 'MUR', 'INTERIEUR', 'BANCHE'],
       ['GO', 'MUR', 'INTERIEUR', 'BANCHE'],
       ...,
       ['GO', 'MUR', 'INTERIEUR', 'BANCHE'],
       ['GO', 'MUR', 'INTERIEUR', 'BANCHE'],
       ['GO', 'MUR', 'INTERIEUR', 'BANCHE']], dtype=object)

In [8]:
y_pred_df = pd.DataFrame(y_pred, columns=targets)

In [9]:
y_pred_df['013ec_localisation'].value_counts()

INTERIEUR    1170
EXTERIEUR      92
CES             4
Name: 013ec_localisation, dtype: int64

In [ ]:
# Mettre les prédictions dans un DataFrame
df_pred = pd.DataFrame(y_pred, columns=targets)

# Afficher les premières lignes
print(df_pred.head())

# Sauvegarder si besoin
df_pred.to_csv("resultats_predictions_murs6.csv", index=False)

  011ec_lot 012ec_ouvrage 013ec_localisation 014ec_mode_constructif
0        GO           MUR          INTERIEUR                 BANCHE
1        GO           MUR          INTERIEUR                 BANCHE
2        GO           MUR          INTERIEUR                 BANCHE
3        GO           MUR          INTERIEUR                 BANCHE
4        GO           MUR          INTERIEUR                 BANCHE
